# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes (Town=Hartford, last five years): https://www.ctcrash.uconn.edu/QueryTool2.action

This notebook combines data from two tables (0 and 2) into a small JSON file, which is an array of the following objects:
```
{
    "x":41.73936,
    "y":-72.66404,
    "d":1593820800,
    "p":0,
    "c":0
}
```

`d` is timestamp, `p` is a flag for pedestrians (1=true, 0=false), `c` is a flag for cyclists.

In [7]:
import pandas as pd

In [8]:
export = '70836'

### Import roles from table 2

In [9]:
roles = pd.read_csv(
    f'export_{export}/export_{export}_2.csv',
    skiprows=1,
    usecols=['CrashId', 'Person Type Text Format']
)

roles = roles.groupby(['CrashId'])['Person Type Text Format'].unique()

### Import crashes from table 0

In [10]:
crashes = pd.read_csv(
    f'export_{export}/export_{export}_0.csv',
    skiprows=1,
    usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash']
)

### Merge data and save to .json

In [11]:
crashes.columns = ['id', 'x', 'y', 'd']

crashes.d = crashes.d.apply(lambda x: int(pd.to_datetime(x).timestamp()))

crashes['p'] = crashes.id.apply(lambda x: 1 if x in roles and 'Pedestrian' in roles[x] else 0)
crashes['c'] = crashes.id.apply(lambda x: 1 if x in roles and 'Bicyclist' in roles[x] else 0)

In [12]:
crashes.loc[:, 'x': 'c'].to_json('data/crashes.json', orient='records', double_precision=5)